# OpenStreetMap Data Data Audit
___

Matthew Flood

### Map Area
 New Orleans, LA, United States

- [https://www.openstreetmap.org/relation/131885](https://www.openstreetmap.org/relation/131885)
- [https://www.openstreetmap.org/export#map=11/30.0326/-89.8826](https://www.openstreetmap.org/export#map=11/30.0326/-89.8826)
- [https://overpass-api.de/api/map?bbox=-90.2170,29.8633,-89.5482,30.2015](https://overpass-api.de/api/map?bbox=-90.2170,29.8633,-89.5482,30.2015)

I'll be visiting New Orleans in the next month, so I want to get a feel for the amenities and sights.
   
   

In [1]:
import xml.etree.cElementTree as ET
import pprint
from collections import defaultdict
import re
MAP_PATH="maps/new_orleans_city.osm"


## Audit the top level tags

> First, we want to identify all the top level tags in the data set to make sure the document looks sane from a high level.

In [2]:
def identify_tags_fullpath(filepath):
    """
        Identifies tags along with their parent heirarchy
    """
    tag_types = {}
    tag_types_ordered = []
    stack = []
    events = {}
    with open(filepath, "r") as handle:
        for event, elem in ET.iterparse(handle, events=("start","end")):
            if event == "start":
                stack.append(elem.tag)
                tag_path = "/".join(stack)
                tag_types.setdefault(tag_path, 0)
                if tag_types[tag_path] == 0:
                    # print(tag_path)
                    tag_types_ordered.append(tag_path)
                tag_types[tag_path] += 1
            if event == "end":
                stack.pop()

    for item in tag_types_ordered:
        print("{0: <20} {1: >10}".format(item, tag_types[item]))

In [3]:
identify_tags_fullpath(MAP_PATH)

osm                           1
osm/note                      1
osm/meta                      1
osm/bounds                    1
osm/node                1786106
osm/node/tag             130282
osm/way                  189487
osm/way/nd              2050430
osm/way/tag              535711
osm/relation               1407
osm/relation/member       25395
osm/relation/tag           7114
osm/remark                    1


> This structure is what we expected and nothing looks out of place.  We see &lt;node&gt; and &lt;way&gt; tags.  Those both have nested &lt;tag&gt; elements, and &lt;way&gt; tags also have nested &lt;nd&gt; tags.

## Audit the tag 'keys' 

> Now we want to audit the "k" value for each "&lt;tag&gt;" and see if there are any potential problems.  The following utility function will put each key into one of four buckets.

In [4]:
def audit_tag_keys(filepath, parent_tag):
    """
        parent_tag: way, node, relation
        
        Audit all the "k" attributes and put them into 4 buckets
        1. all lowercase and valid
        2. all lowercase with a ':' in the middle, and valid
        3. ones with bad characters
        4. others (none of the above)
    """
    
    lowercase_tags = {}
    lowercase_colon_tags = {}
    bad_char_tags = {}
    other_tags = {}
   
    regex_all_lower = re.compile(r'^([_a-z])*$')
    lower_colon = re.compile(r'^([_a-z])*:([_a-z])*$')
    problemchars = re.compile(r'[^_a-z]')
    
    with open(filepath, "r") as handle:
        for event, elem in ET.iterparse(handle, events=("start",)):
            if elem.tag == parent_tag:
                for elem in elem.iterfind("tag"):
                    if elem.tag == "tag":
                        k_val = elem.attrib['k']
                        v_val = elem.attrib['v']
                        # print(k_val)
                        if regex_all_lower.match(k_val):
                            lowercase_tags.setdefault(k_val, set())
                            lowercase_tags[k_val].add(v_val)
                        elif lower_colon.match(k_val):
                            lowercase_colon_tags.setdefault(k_val, set())
                            lowercase_colon_tags[k_val].add(v_val)
                        elif problemchars.match(k_val):
                            bad_char_tags.setdefault(k_val, set())
                            bad_char_tags[k_val].add(v_val)
                        else:
                            other_tags.setdefault(k_val, set())
                            other_tags[k_val].add(v_val)
    
    return (lowercase_tags, lowercase_colon_tags, bad_char_tags, other_tags)

> I want to look at Nodes and Ways separately, so I'm going to store "key" info for each of thoise in separate variables

In [5]:
node_lowercase_tags, node_lowercase_colon_tags, node_bad_char_tags, node_other_tags = audit_tag_keys(MAP_PATH, "node")
way_lowercase_tags, way_lowercase_colon_tags, way_bad_char_tags, way_other_tags = audit_tag_keys(MAP_PATH, "way")


### tags with bad characters
First, let's look a the keys with bad characters in nodes, and then ways

In [6]:
sorted_keys = list(node_bad_char_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_bad_char_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

ADDRESS_LA                   85 9330 FORSHEY ST, 3135 CHERRY ST, 3229 MISTLETOE ST, 9015 FORSHEY ST, 9103 FIG ST
FIXME                         9 can be amenity=restaurant, can't verified, location should be in Algiers, Find o
HIV                           1 Delgado STD Clinic
Payments                      1 MasterCard, Amex, Cash, Visa, Discover, Checks


In [7]:
sorted_keys = list(way_bad_char_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_bad_char_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

ADDRESS_LA                   13 3018 CHERRY ST, 3212 HAMILTON ST, 9414 COLAPISSA ST, 3321 HAMILTON ST, 9308 OLIV
FIXME                         6 rough alignment in places, Need a name, dual carriageway, check lanes, verify, r
FIXME:ncn_ref                 1 verify
FIXME:ref                     1 verify north end
NHD:ComID                  6845 139202506, 139148528, 139149579, 139142741, 139182351, 139182891, 139148354, 139
NHD:Elevation                 5 -3.00000000000, -2.70000000000, 0.90000000000, -2.40000000000, 0.60000000000
NHD:FCode                    13 46600, 48300, 39800, 43612, 39009, 46006, 36900, 43609, 39001, 33600, 39004, 445
NHD:FDate                    12 2005/12/05, 2005/01/14, 2006/09/25, 2009/01/20, 2006/09/22, 2005/08/24, 2005/12/
NHD:FTYPE                     7 SwampMarsh, CanalDitch, SeaOcean, Lock Chamber, Reservoir, StreamRiver, LakePond
NHD:FType                     5 CanalDitch, Lock Chamber, Gate, StreamRiver, Wall
NHD:GNIS_ID                  12 01628171, 0

> So, these keys do not really have bad data, they just have capital letters.  And some, like `NHD:FTYPE` and `NHD:FType` have different capitlization.
> We will address these by forcing all the k values to lowercase when we perform our import


### Other keys

> Now we want to look at the "other" bucket to see what kind of issues there are.

In [8]:
sorted_keys = list(node_other_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_other_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

amenity_1                     1 cafe
currency:ETH                  1 yes
currency:LTC                  1 yes
currency:XBT                  1 yes
generator:output:electricity          1 560 MW
gnis:Class                    1 Populated Place
gnis:County                   5 Orleans, Jefferson, St. Bernard, St. Tammany, Plaquemines
gnis:County_num               5 087, 071, 103, 051, 075
gnis:ST_alpha                 1 LA
gnis:ST_num                   1 22
name:etymology:wikidata          1 Q8027
name_1                        1 Lakefront Airport
service:bicycle:diy           1 yes
service:bicycle:pump          1 yes
service:bicycle:rental          1 yes
service:bicycle:repair          1 yes
service:bicycle:retail          1 yes
service:bicycle:second_hand          1 yes
service:bicycle:tools          1 yes
socket:nema_5_20              1 2
socket:type1                  2 2, 1
source:name:oc                1 Lo Congrès
wikipedia_1                   1 pl:Pomnik Andrew Jacksona w Waszyngtonie


In [9]:
sorted_keys = list(way_other_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_other_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

building:roof:material          1 glass
cycleway:left:oneway          1 no
cycleway:left:width           1 9'
cycleway:right:oneway          1 no
cycleway:right:width          5 9, 9', 11', 12', 8'
destination:ref:lanes          2 LA 45, LA 45|LA 45
gnis:Class                    1 Populated Place
gnis:County                   1 Jefferson
gnis:County_num               1 051
gnis:ST_alpha                 1 LA
gnis:ST_num                   1 22
name:etymology:wikidata          1 Q8027
name_1                       93 Munster Drive, Harrow Road, State Route 3018, Prytania Street, State Route 3017,
name_2                        7 Louisiana Hwy 406, West Saint Jean Baptiste Street, Claiborne Avenue, Sanctuary 
plant:output:electricity          2 1646 MW, 959 MW
ref:AEFE                      1 272N15
service:vehicle:body_repair          1 yes
sidewalk:both:surface          1 concrete
tiger:name_base_1           119 Carlisle, Villere, State Route 3018, Mink, State Route 48, State Route 3017, St

> Again, we see issues related to capitalization.  We also see some numbers, which have to do with the fact that a key can only be used once for a tag, so if you want more than one name, then instead of having three tags with `name`, you need to have `name`, `name_1` and `name_2`.

> To verify that this is the case, I want to look at a few tags that have `name_`.

#### Look into name_1 and amenity_1

>This function will find entities that have a tag with the 
given `tag_key` and print out the full set of tags for the node/way/relation



In [10]:
def print_example_entities(filepath, tag_key, max_results):
    """
        finds and prints up to max_results sample Nodes / Ways / Relations
        that have the specified tag key
    """
    with open(filepath, "r") as handle:
        num_examples_found = 0
        for event, elem in ET.iterparse(handle, events=("start",)):
            if elem.tag in ("node", "relation", "way"):
                found_an_example = False
                example_dict = {}
                
                for child_tag in elem.iterfind("tag"):
                    if child_tag.tag == "tag":
                        k_val = child_tag.attrib['k']
                        v_val = child_tag.attrib['v']
                        
                        example_dict.setdefault(k_val, set())
                        example_dict[k_val].add(v_val)
                        
                        if k_val == tag_key:
                            found_an_example = True
                
                if found_an_example:
                    num_examples_found += 1
                    print("Example {}".format(elem.tag))    
                    pprint.pprint(example_dict)
                          
                    if max_results and (num_examples_found >= max_results):
                        # we have printed max_results so we are done
                        return
 

In [12]:
print_example_entities(MAP_PATH, "name_1", max_results=3)

Example node
{'addr:state': {'LA'},
 'aeroway': {'aerodrome'},
 'closest_town': {'New Orleans, Louisiana'},
 'ele': {'1'},
 'faa': {'NEW'},
 'gnis:county_name': {'Orleans'},
 'gnis:created': {'08/01/1991'},
 'gnis:feature_id': {'1632892'},
 'gnis:feature_type': {'Airport'},
 'iata': {'NEW'},
 'icao': {'KNEW'},
 'name': {'New Orleans Lakefront Airport'},
 'name:en': {'New Orleans Lakefront Airport'},
 'name_1': {'Lakefront Airport'},
 'operator': {'Orleans Levee District'},
 'source': {'wikipedia'},
 'source_ref': {'geonames.usgs.gov'},
 'type': {'public'},
 'wikidata': {'Q10853543'},
 'wikipedia': {'en:New Orleans Lakefront Airport'}}
Example way
{'highway': {'secondary'},
 'maxspeed': {'45 mph'},
 'name': {'Peters Road'},
 'name_1': {'State Route 3017'},
 'ref': {'LA 3017'},
 'tiger:cfcc': {'A41;A35'},
 'tiger:county': {'Jefferson, LA'},
 'tiger:name_base': {'Peters'},
 'tiger:name_base_1': {'State Route 3017'},
 'tiger:name_type': {'Rd'},
 'tiger:reviewed': {'no'},
 'tiger:zip_left':

> Yup - these all have name_1 as alternates to the name key. I want to look at amenity_1 as well

In [13]:
print_example_entities(MAP_PATH, "amenity_1", max_results=3)

Example node
{'addr:city': {'New Orleans'},
 'addr:housenumber': {'2372 #130'},
 'addr:postcode': {'70117'},
 'addr:state': {'LA'},
 'addr:street': {'Saint Claude Avenue'},
 'amenity': {'restaurant'},
 'amenity_1': {'cafe'},
 'delivery': {'no'},
 'name': {'The Spotted Cat'},
 'takeaway': {'yes'}}


### Lowercase keys without semicolons 

> Now we want to look at the lowercase keys, the ones without semicolons.

In [14]:
sorted_keys = list(way_lowercase_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_lowercase_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

access                        7 destination, permissive, customers, delivery, private, no, yes
admin_level                   2 8, 6
aeroway                       6 terminal, taxiway, helipad, runway, hangar, apron
alt_name                     15 Walmart Supercenter New Orleans, 2 Canal Street, Audubon Butterfly Garden and In
amenity                      48 dentist, marketplace, pub, dive_centre, toilets, arts_centre, parking, bar, clin
architect                     4 Edward Durell Stone, Charles Moore, DMJM, Emile Weil
area                          2 no, yes
artist_name                   1 Multiple
artwork_type                  2 statue, sculpture
atm                           1 yes
attribution                   1 NHD
backrest                      1 yes
barrier                       9 gate, kerb, fence, guard_rail, retaining_wall, wire_fence, dyke, wall, hedge
bicycle                       4 permissive, no, yes, designated
boat                          2 no, yes
border_type            

In [15]:
sorted_keys = list(node_lowercase_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_lowercase_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

access                        5 permissive, customers, private, no, yes
aeroway                       3 aerodrome, holding_position, helipad
alt_name                     19 Lou!s!ana, Hotel St. Helene, Fulton Place Parking Center, Jesuit Church, Optical
amenity                      64 kindergarten, dentist, marketplace, pub, vending_machine, toilets, arts_centre, 
artist                        1 Clark Mills
artist_name                  14 Madeleine Faust, Paul Perret, Enrique Alferez, Franco Alessandrini, Yaacov Agam,
artwork_type                  4 statue, mural, painting, sculpture
atm                           2 no, yes
attribution                   1 USGS 2001 County Boundary
automated                     1 yes
backrest                      1 yes
barrier                       8 gate, bollard, block, entrance, lift_gate, guard_rail, swing_gate, yes
beauty                        3 nails, hair_removal, spa
bench                         2 no, yes
bicycle                       2 no, yes

> Nothing seems out of place there.  

# Lowercase keys with semicolons

Finally, we want to look at the ones with semicolons

In [16]:
sorted_keys = list(way_lowercase_colon_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = way_lowercase_colon_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

abandoned:railway             1 rail
abandoned:tourism             1 theme_park
addr:city                    16 Gretma, new orleans, Gretna, Jefferson, Metairie, Harahan, Marrero, New Orleans,
addr:country                  2 UA, US
addr:full                     2 99 Westbank Expy, 6000 Bullard Ave
addr:housename                2 Arabella Station, Bagneris Home
addr:housenumber           8908 6447, 4155, 5329, 2776, 2214, 3403, 1107, 13937, 6477, 13027, 5451, 8649, 9239, 
addr:postcode                30 70114, 70056, 70115, 70118, 70128, 70131, 70003, 70124, 70135, 70123, 70126, 700
addr:state                    4 Louisiana, la, lA, LA
addr:street                1998 Annunciation Street, Haydel Drive, Race Street, Macarthur Boulevard, Treme Stree
addr:unit                     1 115
amenity:bar                   1 yes
amenity:historic              1 hospital
brand:wikidata               32 Q1141226, Q1047239, Q524629, Q154950, Q2078880, Q1591889, Q2645636, Q7500392, Q1
brand:wikipedia   

In [17]:
sorted_keys = list(node_lowercase_colon_tags.keys())
sorted_keys.sort()
for item in sorted_keys:
    value_set = node_lowercase_colon_tags[item]
    print("{0: <20} {1: >10} {2:}".format(item, len(value_set), ", ".join(list(value_set))[:80] ) )

abandoned:man_made            1 lighthouse
addr:city                    17 New Orleans LA, Westwego, new orleans, Harvey, Elmwood, Metairie, Harahan, Marre
addr:country                  1 US
addr:full                     6 4001 Behrman Pl, 4810 Lapalco Blvd, 4301 Chef Menteur Highway, 5110 Jefferson Hw
addr:housename                9 Aframe, Chase Bank, Patio, Fritzel's - Traditional Jazz Club, 1555 Poydras, Malb
addr:housenumber           6685 4155, 987, 5329, 2776, 2214, 3403, 1107, 6477, 5451, 2208, 4063, 2848, 255, 9135
addr:interpolation            2 600, 200
addr:place                    1 Suite 500
addr:postcode                37 70125, 70114, 70056, 70115, 70118, 70128, 70005, 70012, 70003, 70170, 70124, 701
addr:state                    3 Louisiana, La, LA
addr:street                1294 Annunciation Street, Boeing Street, Race Street, 621 Canal Street, Neron Place, 
addr:unit                    31 Suite 2, Suite 100, Suite 250, Suite 29-2959, 100, G, #100, Suite A-1, 119, 11,

### Auditing Street Names

> Now I want to look at street names to make sure they are consistent

> First we'll build a function that uses regexes to identiyfy "normal" street addresses.  We'll pass in an argument to flip between showing normal and irregular addresses.


In [24]:
def print_streets(show_normal=True):
    """
        look at address:street values
        that match (or don't match) a street regex.
    """
    map_path = MAP_PATH
    tag_key = "addr:street"
    
    value_counts = {}
    with open(map_path, "r") as handle:
        
        for event, elem in ET.iterparse(handle, events=("start",)):
            if elem.tag in ("node", "relation", "way"):
                
                value_dict = {}
                
                for child_tag in elem.iterfind("tag"):
                    k_val = child_tag.attrib['k']
                    v_val = child_tag.attrib['v']
                    
                    if k_val == tag_key:
                        value_counts.setdefault(v_val, 0)
                        value_counts[v_val] += 1
    
    return value_counts


In [25]:
value_counts = print_streets()

In [26]:
# print the values, along with the count of occurrences
sorted_keys = list(value_counts.keys())
sorted_keys.sort()
for item in sorted_keys:
    count = value_counts[item]
    print("{0: <20} {1: >10}".format(item, count) )


10th Street                  56
1227 Tulane Ave               1
12th Street                  33
14th Street                  49
16th Street                  40
18th Street                  51
1st Street                  366
20th Street                  38
22nd Street                  39
26th Street                  37
28th Street                  34
2nd Street                  368
30th Street                  37
3157 Gentilly Blvd #2019          1
32nd Street                  10
33rd Street                  17
34th Street                  21
35th Street                  13
36th Street                  17
37th Street                   9
38th Street                  16
39th Street                  17
3rd Street                  403
40th Street                  22
4th Street                  369
5th Street                    1
621 Canal Street              1
6th Street                  267
7th Street                  269
8th Street                  173
9th Street                   88
A P 

Thornley Drive               20
Thrasher Street              23
Thrush Street                 6
Tilbury Road                 36
Tilford Road                 24
Timber Bluff Lane            47
Timber Court                 13
Timber Crest Drive           38
Timber Haven Lane            47
Timber Ridge Court            4
Timber Wolf Lane             39
Timberlane Drive              1
Timoleon Street              24
Tita Street                  69
Todd Place                    5
Toledano Street             344
Topaz Street                 81
Torrey Pines Drive           10
Toulon Street                57
Toulouse St                   1
Toulouse Street             286
Touro Street                410
Townsend Place               18
Trade Winds Court            12
Trafalgar Street             55
Transcontinental Drive          1
Trapier Avenue               62
Trappers Court               18
Treasure Street             101
Treme Street                 39
Treves Street                51
Triano

In [100]:
street_type_re = re.compile(r'(.*) (\b\S+\.?)$', re.IGNORECASE)

# I built this list up as I looked at the results
expected = ["Street",
            "Avenue",
            "Court",
            "Drive",
            "Road",
            "Boulevard",
            "Place",
            "Lane",
            "Circle",
            "Cove",
            "Arcade",
            "Way",
            "Walk",
            "Highway",
            "Parkway",
            "Alley",
            "Plaza"]

to_fix = {"St": "Street",
          "St.": "Street",
          "Ave": "Avenue",
          "Pkwy": "Parkway",
          "Hwy": "Highway",
          "Blvd": "Boulevard",
          "Dr": "Drive",
          "Ave.": "Avenue",
          "Pky": "Parkway"}

fullname_mapping = {"Banks": "Banks Street",
                    "Regent Street #C": "Regent Street",
                    "Dumaine": "Dumaine Street",
                    "Magazine Street;Magazine Street": "Magazine Street",
                    "Magazine": "Magazine Street",
                    "Rosa PARK": "Rosa Park",
                    "Severn": "Severn Avenue",
                    "St. Peter": "Saint Peter Street",
                    "3157 Gentilly Blvd #2019": "Gentilly Boulevard",
                    "Marais": "Marais Street",
                    "Royal": "Royal Street",
                    "Canal Street at Bourbon": "Canal Street",
                    "General Taylor": "General Taylor Street",
                    "Gretna Blvd Ste A": "Gretna Boulevard",
                    "Manhattan Boulevard Building": "Manhattan Boulevard",
                    "Saint Charles Avenue, Suite 114-351": "Saint Charles Avenue",
                   }

sorted_keys = list(value_counts.keys())
sorted_keys.sort()
for item in sorted_keys:
    
    # patch full items that are borked
    item = fullname_mapping.get(item, item) 
    match = street_type_re.match(item)
    if match:
        #continue
        first_path = match.group(1)
        street_type = match.group(2)
        street_type = to_fix.get(street_type, street_type)
            
        if street_type not in expected:
            print("{0: <20} '{1:}' '{2:}'".format(item, match.group(1), match.group(2)) )
        else:
            pass
            print("Fixed Street: '{} {}'".format(first_path, street_type))
    else:
        print("********************** Special {}".format(item))
        

# UPDATE THIS VARIABLE
mapping = { "St": "Street",
            "St.": "Street"
            }

Fixed Street: '10th Street'
Fixed Street: '1227 Tulane Avenue'
Fixed Street: '12th Street'
Fixed Street: '14th Street'
Fixed Street: '16th Street'
Fixed Street: '18th Street'
Fixed Street: '1st Street'
Fixed Street: '20th Street'
Fixed Street: '22nd Street'
Fixed Street: '26th Street'
Fixed Street: '28th Street'
Fixed Street: '2nd Street'
Fixed Street: '30th Street'
Fixed Street: 'Gentilly Boulevard'
Fixed Street: '32nd Street'
Fixed Street: '33rd Street'
Fixed Street: '34th Street'
Fixed Street: '35th Street'
Fixed Street: '36th Street'
Fixed Street: '37th Street'
Fixed Street: '38th Street'
Fixed Street: '39th Street'
Fixed Street: '3rd Street'
Fixed Street: '40th Street'
Fixed Street: '4th Street'
Fixed Street: '5th Street'
Fixed Street: '621 Canal Street'
Fixed Street: '6th Street'
Fixed Street: '7th Street'
Fixed Street: '8th Street'
Fixed Street: '9th Street'
Fixed Street: 'A P Tureaud Avenue'
Fixed Street: 'Abalon Court'
Fixed Street: 'Abbey Drive'
Fixed Street: 'Aberdeen Road'


Fixed Street: 'Wadsworth Drive'
Fixed Street: 'Wagner Street'
Fixed Street: 'Wainwright Drive'
Fixed Street: 'Wakefield Place'
Fixed Street: 'Waldo Drive'
Fixed Street: 'Wales Street'
Fixed Street: 'Walker Street'
Fixed Street: 'Wall Boulevard'
Fixed Street: 'Wallace Drive'
Fixed Street: 'Walmsley Avenue'
Fixed Street: 'Walnut Street'
Fixed Street: 'Walter Street'
Fixed Street: 'Warbler Street'
Fixed Street: 'Warfield Drive'
Fixed Street: 'Warfield Street'
Fixed Street: 'Warren Drive'
Fixed Street: 'Warrington Drive'
Fixed Street: 'Warrington Place'
Fixed Street: 'Warwick Court'
Fixed Street: 'Washington Avenue'
Fixed Street: 'Waterford Boulevard'
Fixed Street: 'Wave Drive'
Fixed Street: 'Waverly Drive'
Fixed Street: 'Wayfarer Street'
Fixed Street: 'Wayside Drive'
Fixed Street: 'Weaver Avenue'
Fixed Street: 'Webster Street'
Fixed Street: 'Weiblen Place'
Fixed Street: 'Wellington Avenue'
Fixed Street: 'Wentworth Drive'
Fixed Street: 'Werner Drive'
Fixed Street: 'West Adams Court'
Fixed 